In [1]:
import os
import pandas as pd
import scipy
import numpy as np
import anndata
import snapatac2 as snap

In [2]:
## Some little functions to deal with adding to ads obs (maybe I don't understand the API but it doesn't act like AnnData in memory?)
def fill_list(index,d,fill='nan'):
    '''like df.column.map(d) but for a list'''
    out=[]
    for x in index:
        try:
            out.append(d[x])
        except:
            out.append(fill)
    return(out)

def add_to_anndataset_obs(df,columns,index_col,ads,ads_index_col,fill='nan'):
    '''annotation dataframe, columns to add, index column from annot dataframe, anndatasets, corresponding index_col in ads'''
    for c in columns:
        d=dict(zip(df[index_col],df[c]))
        ads.obs[c]=fill_list(ads.obs[ads_index_col],d,fill='nan')

# Group anndatas into AnnDataSet

In [ ]:
import os
import snapatac2 as snap
import anndata as ad

## Define the root directory to start walking
root_directory = "/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/BasalGanglia/Macaque/AIT117"

## Walk through the directory tree, assumes "ATAC" is the root directory containing all the snapatac2 h5ad files
h5ad_files = {}
for root, dirs, files in os.walk(root_directory):
    # Check if 'atac' is in the current directory name
    if 'ATAC' in root:
        for f in files:
            try:
                # Check if the file is an h5ad file
                if f.endswith('.h5ad') & (not '.h5ads' in f) & (not '_nodoub' in f):
                    file_path = os.path.join(root, f)
                    print(f'Reading file: {file_path}')
                    # Read the h5ad file
                    adata = snap.read(file_path, backed='r+')
                    print(adata)
                    ##
                    h5ad_files[f] = adata
            except:
                #pass
                adata.close()
                print('problem')
 
## Check if we have h5ad files to concatenate
if h5ad_files:
    try:
        # Concatenate all the h5ad files
        concatenated_adata = snap.AnnDataSet(
            adatas=[(k, h5ad_files[k]) for k in h5ad_files],
            filename=os.path.join("/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/BasalGanglia/Macaque/AIT117/ATAC/h5ads", 'macaque_BG_AIT117.h5ads')
        )
        concatenated_adata.uns["species_name"] = "Macaque"
        concatenated_adata.close()
    except:
        print("Failed")
